# 3D-CODED

In [1]:
import numpy as np
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
from tqdm import tqdm
import metrics.geodist_metric as geodist_metric
from utils.shape_util import compute_geodesic_distmat
import torch


def get_geo_err_3dcoded(dataset_name):

    single_dataset, pair_dataset = data_loading.get_val_dataset(
        dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
    )

    dist_mat_list = []

    for i in tqdm(range(len(single_dataset))):

        data_i = single_dataset[i]

        dist_mat = torch.tensor(
            compute_geodesic_distmat(data_i['verts'].numpy(), data_i['faces'].numpy())    
        )
        
        dist_mat_list.append(dist_mat)

    path_3dc = f'/lustre/mlnvme/data/s94zalek_hpc-shape_matching/results_baselines/3D_CODED/{dataset_name}'

    geo_err_list = []

    for i in tqdm(range(len(pair_dataset))):
        
        data_i = pair_dataset[i]
        
        first_idx = data_i['first']['id']
        second_idx = data_i['second']['id']
        
        p2p_3dc = torch.tensor(
            np.loadtxt(f'{path_3dc}/{first_idx}-{second_idx}.txt')
        ).int()
        
        dist_x = dist_mat_list[first_idx]
        dist_y = dist_mat_list[second_idx]
        
        corr_first = data_i['first']['corr']
        corr_second = data_i['second']['corr']
        
        # geo_err = geodist_metric.calculate_geodesic_error(
        #     dist_x, corr_first.cpu(), corr_second.cpu(), p2p_3dc, return_mean=True
        # ) * 100
        
        geo_err = geodist_metric.calculate_geodesic_error(
            dist_y, corr_second.cpu(), corr_first.cpu(), p2p_3dc, return_mean=True
        ) * 100
        
        geo_err_list.append(geo_err)
        
    geo_err_list = torch.tensor(geo_err_list)
    print(f'{dataset_name}, mean geo err: {geo_err_list.mean()}, median: {geo_err_list.median()}')

# Transmatch

In [2]:
dataset_name = 'FAUST_r_pair'

single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
)

dist_mat_list = []

for i in tqdm(range(len(single_dataset))):

    data_i = single_dataset[i]

    dist_mat = torch.tensor(
        compute_geodesic_distmat(data_i['verts'].numpy(), data_i['faces'].numpy())    
    )
    
    dist_mat_list.append(dist_mat)

In [4]:
def get_geo_err_transmatch(dataset_name):
    single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
    )

    dist_mat_list = []

    for i in tqdm(range(len(single_dataset))):

        data_i = single_dataset[i]

        dist_mat = torch.tensor(
            compute_geodesic_distmat(data_i['verts'].numpy(), data_i['faces'].numpy())    
        )
        
        dist_mat_list.append(dist_mat)



    path = f'/home/s94zalek_hpc/baselines/transmatching/evaluation/predictions/{dataset_name}/our(ckp=s2s_weighted_bary,refine=True,area_norm=True,refine_steps=100)'

    geo_err_list = []

    for i in tqdm(range(len(pair_dataset))):
        
        data_i = pair_dataset[i]
        
        first_idx = data_i['first']['id']
        second_idx = data_i['second']['id']
        
        p2p = torch.tensor(
            np.loadtxt(f'{path}/{i:03d}/pred_matching_A_to_B.txt')
        ).to(torch.int64)
        
        # print(f'{path}/{i:03d}/pred_matching_A_to_B.txt')
        
        dist_x = dist_mat_list[first_idx]
        dist_y = dist_mat_list[second_idx]
        
        corr_first = data_i['first']['corr']
        corr_second = data_i['second']['corr']
        
        # geo_err = geodist_metric.calculate_geodesic_error(
        #     dist_x, corr_first.cpu(), corr_second.cpu(), p2p, return_mean=True
        # ) * 100
        
        geo_err = geodist_metric.calculate_geodesic_error(
            dist_y, corr_second.cpu(), corr_first.cpu(), p2p, return_mean=True
        ) * 100
        
        geo_err_list.append(geo_err)
        
        # break
        
    geo_err_list = torch.tensor(geo_err_list)
    print(f'{dataset_name}, mean geo err: {geo_err_list.mean()}, median: {geo_err_list.median()}')

In [5]:
for dataset_name in [
        'FAUST_r_pair',
        'FAUST_a_pair', 
        'SCAPE_r_pair',
        'SCAPE_a_pair',
        'SHREC19_r_pair'
        ]:
    get_geo_err_transmatch(dataset_name)

# ConsistentFmaps

In [ ]:

path = f'/home/s94zalek_hpc/baselines/Spatially-and-Spectrally-Consistent-Deep-Functional-Maps/data/results/{dataset_name[:-5]}/p2p_21'

geo_err_list = []

for i in tqdm(range(len(pair_dataset))):
    
    data_i = pair_dataset[i]
    
    first_idx = data_i['first']['id']
    second_idx = data_i['second']['id']
    
    p2p = torch.tensor(
        np.loadtxt(f'{path}/{first_idx}_{second_idx}.txt')
    ).int()
    
    dist_x = dist_mat_list[first_idx]
    dist_y = dist_mat_list[second_idx]
    
    corr_first = data_i['first']['corr']
    corr_second = data_i['second']['corr']
    
    geo_err = geodist_metric.calculate_geodesic_error(
        dist_x, corr_first.cpu(), corr_second.cpu(), p2p, return_mean=True
    ) * 100
    
    # geo_err = geodist_metric.calculate_geodesic_error(
    #     dist_y, corr_second.cpu(), corr_first.cpu(), p2p, return_mean=True
    # ) * 100
    
    geo_err_list.append(geo_err)
    
    # print(geo_err)
    # break
    
geo_err_list = torch.tensor(geo_err_list)
print(f'{dataset_name}, mean geo err: {geo_err_list.mean()}, median: {geo_err_list.median()}')

# Simplified ...

In [ ]:
import pickle


path = f'/home/s94zalek_hpc/baselines/SimplifiedFmapsLearning/res_cache/train_faust/test_shrec19/maps'
# path = f'/home/s94zalek_hpc/baselines/SimplifiedFmapsLearning/res_cache/train_faust/test_faust_24-11-04_17-06-01/maps'
# path = f'/home/s94zalek_hpc/baselines/SimplifiedFmapsLearning/res_cache/train_faust/test_scape_24-11-04_17-25-39/maps'

geo_err_list = []

for i in tqdm(range(len(pair_dataset))):
    
    # i = 4
    
    data_i = pair_dataset[i]
    
    first_idx = data_i['first']['id']
    second_idx = data_i['second']['id']
    
    off_file_first = single_dataset.off_files[first_idx]
    off_file_second = single_dataset.off_files[second_idx]

    # from data/FAUST_r/off/tr_reg_080.off get tr_reg_080
    first_off_name = off_file_first.split('/')[-1].split('.')[0]
    second_off_name = off_file_second.split('/')[-1].split('.')[0]
    
    pickled_file = f'{path}/{second_off_name}-{first_off_name}.p'   
    with open(pickled_file, 'rb') as f:
        data = pickle.load(f)
        
    p2p = data['pmap10_ref'].cpu()
        
    # print(data)
    # break
    
    # p2p = torch.tensor(
    #     np.loadtxt(f'{path}/{first_idx}_{second_idx}.txt')
    # ).int()
    
    dist_x = dist_mat_list[first_idx]
    dist_y = dist_mat_list[second_idx]
    
    corr_first = data_i['first']['corr']
    corr_second = data_i['second']['corr']
    
    # geo_err = geodist_metric.calculate_geodesic_error(
    #     dist_x, corr_first.cpu(), corr_second.cpu(), p2p, return_mean=True
    # ) * 100
    
    geo_err = geodist_metric.calculate_geodesic_error(
        dist_y, corr_second.cpu(), corr_first.cpu(), p2p, return_mean=True
    ) * 100
    
    geo_err_list.append(geo_err)
    
    # print(geo_err)
    # break
    
geo_err_list = torch.tensor(geo_err_list)
print(f'{dataset_name}, mean geo err: {geo_err_list.mean()}, median: {geo_err_list.median()}')

FAUST_a_pair, mean geo err: 12.689779487620378, median: 5.11637879633151

SHREC19_r_pair, mean geo err: 6.349553541101691, median: 4.081554091554718

SCAPE_a_pair, mean geo err: 8.630594943047253, median: 2.7107606137821887

In [ ]:
geo_err_list

# Visualization

In [ ]:
import trimesh

scene = trimesh.Scene()

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

# i = 1
# data_i = pair_dataset[i]

# first_idx = data_i['first']['id']
# second_idx = data_i['second']['id']

# p2p = torch.tensor(
#     # np.loadtxt(f'{path}/{i:03d}/pred_matching_A_to_B.txt')
#     np.loadtxt('/home/s94zalek_hpc/baselines/transmatching/evaluation/predictions/FAUST_r_pair/our(ckp=s2s_weighted_bary,refine=True,area_norm=True,refine_steps=100)/001/pred_matching_A_to_B.txt')
# ).to(torch.int64)
    

plotting_utils.plot_p2p_map(
    scene,
    
    
    data_i['second']['verts'], data_i['second']['faces'],
    data_i['first']['verts'], data_i['first']['faces'],
    
    p2p,
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()